# Experiment settings

Dataset: KOSPI index from Jan, 2011 to Dec, 2020\
Included data in default: Daily log return, Daily trading range, Log volume change\

# Data analysis

- Augment Dickey Fuller test -> check for presence of unit root of the daily log return.
ADF test statistics value: -18.6316
p-value: 2.0573e-30

Therefore, the daily log return is stationary.

## Model selection part


- GARCH model selection

GARCH model with different lags

For selecting appropriate lags for GARCH models, partial autocorrelation function is used

![title](images/kospi_pacf.png)

Candidates: GARCH(1,1), GARCH(2,2), GARCH(3,3), EGARCH(1,1), EGARCH(2,2), EGARCH(3,3)
The candidates are nominated by refering pacf plot. The pacf plot indicates that from lag 1 to lag 3
is out of significance level so that the past 3day values are relatively related to the current
value.

Strategy: The models are fitted by train set. Find the best model among the candidates first using AIC, BIC and Log-Likelihood.
Then, tune the p and q parameters of the chosen one referring to p-value described at the model summary.

AIC, BIC formula: [-2logL + kp] where L is the likelihood function, p is the number of params
k is 2 for AIC, and log(n) for BIC.

Model selection criteria

| Models        | AIC        | BIC       | Log-Likelihood |
| :------------ |:----------:| :--------:| --------------:|
| GARCH(1,1)    | 4789.65    | 4823.13   | -2388.83       |
| GARCH(2,2)    | 4789.29    | 4833.93   | -2386.64       |
| GARCH(3,3)    | 4786.71    | 4842.52   | -2383.36       |
| EGARCH(1,1)   | 4792.16    | 4825.65   | -2390.08       |
| EGARCH(2,2)   | 4790.80    | 4835.45   | -2387.40       |
| EGARCH(3,3)   | 4787.78    | 4843.59   | -2383.89       |

There are no significant difference at AIC, BIC and Log-Likelihood.
So I did all permutations of lags for forecasting.

Results of GARCHs and EGARCHs

All other different permutations of GARCH lags shows the less effectiveness than GARCH(1,1) model
The GARCH(1,1) model has the least errors. Also, EGARCH(1,1) ahs the least errors among them.

| Models        | RMSE       | MAPE      |
| :------------ | :--------: | --------:  |
| GARCH(1,1)    | 0.039      | 0.107     |
| GARCH(2,2)    | 0.041      | 0.113     |
| GARCH(3,3)    | 0.041      | 0.115     |
| GJR-GARCH(1,1)   | 0.0517     | 0.171   |
| GJR-GARCH(2,2)   | 0.0515     | 0.168   |
| GJR-GARCH(3,3)   | 0.0507     | 0.164   |
| EGARCH(1,1)   | 0.0679     | 0.192   |
| EGARCH(2,2)   | 0.0668     | 0.190   |
| EGARCH(3,3)   | 0.0670     | 0.191   |


![title](images/garch11.png)

![title](images/egarch11.png)

Result of LSTM model

Single layer with 10 nodes.

- RMSE: 0.0306
- MAPE: 0.0892
![title](images/single_layer_model_pred.png)

The results are 1 day ahead forecast with 22 rolling window. I am going to do 1 day forecast
with 15 rolling window, and 5 day ahead forecast until this week.
Then will move on write the report.

LSTM model selection:
Single layer: 64 units 0.6  the best batch_size:32, epochs:100

FNN model selection:
3 hidden layers 64,32,20 with tanh activation function
three dropout layers to prevent overfitting which are 0.3, 0.1, 0.5 respectively.


## Experiment Results

### Basic Models: FNN, LSTM

This result is used for benchmarking

22 rolling window -> 1 day ahead forecast

| Models        | Training Error(MAE)   | Testing Error(MSE)   | Testing Error (MAPE) | Testing Error (MAE) |
| :------------ | :--------------: | :------------------: | :-------------------: | ----------------------: |
| LSTM          | 0.0235           | 0.0024               | 0.114                | 0.0267                   |
| FNN           | 0.0380           | 0.0052               | 0.144                | 0.0349                   |
| G-FNN         | 0.0269           | 0.0037               | 0.094                | 0.0253                   |
| R-FNN         | 0.0304           | 0.0044               | 0.109                | 0.0281                   |
| GR-FNN         | 0.0270           | 0.0048               | 0.098                | 0.0281                  |
| G-LSTM         | 0.0165           | 0.0012               | 0.088                | 0.0198                  |
| R-LSTM         | 0.0198           | 0.0016               | 0.116                | 0.0245                  |
| GR-LSTM         | 0.0159           | 0.0018               | 0.098               | 0.0226                  |

## Experiment Overview

There are several settings of this project.

Window = 22 days

| Models        | 5days ahead (MAPE) | 5 days ahead (RMSE)   | 10days ahead (MAPE)  | 10days ahead (RMSE) |  22days ahead (MAPE) | 22days ahead (RMSE) |
| :------------ | :----------------: | :-------------------: | :------------------: | :-------------------: | :------------------: | ------------------: |
| GARCH(1,1)    |            |                |                 |                    |
| GJR-GARCH(1,1)|            |                |                 |                   |
| LSTM          | 0.134              | 0.0430                | 0.149                | 0.0533              | 0.246                | 0.0790              |
| FNN           | 0.118              | 0.0737                | 0.134                | 0.0685              | 0.161                | 0.0781              |
| G-FNN         | 0.107              | 0.0627                | 0.109                | 0.0670              | 0.164                | 0.0747              |
| R-FNN         | 0.107              | 0.0637                | 0.139                | 0.0659              | 0.158                | 0.0780              |
| GR-FNN         | 0.092             | 0.0610                | 0.132                | 0.0705              | 0.171                | 0.0764              |
| G-LSTM         | 0.089             | 0.0509                | 0.103                | 0.0489              | 0.173                | 0.0648              |
| R-LSTM         | 0.113             | 0.0491                | 0.123                | 0.0503              | 0.221                | 0.0727              |
| GR-LSTM         | 0.084            | 0.0453                | 0.111                | 0.0497              | 0.202                | 0.0723              |